Solution by Omkar Kulkarni

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

# Part One

In [4]:
# Function to scrape product listings

def scrape_product_listings(url):
    # Make a request to the URL and parse the HTML content
    file  = open('amazonpage.html', 'r', encoding='utf8')
    response = file.read()
    response
    soup  = BeautifulSoup(response, 'html.parser')
    
    
    # Extract product URLs, names, prices, ratings, and reviews
    product_data = []
    
    search_results = soup.find_all(attrs={'data-component-type':'s-search-result'})
    for product in search_results:
        rating, review = None, None
        name = product.h2.text
        url = 'https://www.amazon.in/'+product.h2.a['href']
        price = product.find(class_='a-offscreen').text
        ASIN = product['data-asin']

        try:
            rating = product.find(class_='a-size-base puis-bold-weight-text').text
        except:
            pass
        try:
            review = product.find(class_='a-size-base s-underline-text').text
        except:
            pass
        
        product_data.append([name, url, price, rating, ASIN])
        

        # Return the data as a DataFrame
        df = pd.DataFrame(product_data, columns=['Names', 'URLs', 'Prices', 'Ratings', 'ASIN'])
        
    return df


url = f'https://www.amazon.in/s?k=bags&page=1'
scrape_product_listings(url)

,Names,URLs,Prices,Ratings,ASIN
0,"SAMRAT Laptop Backpack, Separate Compartment t...",https://www.amazon.in//sspa/click?ie=UTF8&spc=...,₹759,None,B0CFHCPW7V
1,"SAMRAT Laptop Backpack, Separate Compartment t...",https://www.amazon.in//sspa/click?ie=UTF8&spc=...,₹699,None,B0CFDSBXRY
2,Skybags Brat Black 46 Cms Casual Backpack,https://www.amazon.in//Skybags-Brat-Black-Casu...,₹669,4.1,B08Z1HHHTD
3,American Tourister Fizz Large Size 32 Ltrs Cas...,https://www.amazon.in//American-Tourister-AMT-...,"₹1,199",4.0,B07CJCGM1M
4,Half Moon Large 37L Laptop Bag Backpack for me...,https://www.amazon.in//Half-Moon-Backpack-Lugg...,₹849,3.9,B09VCLZ3K4
5,Half Moon 35 Ltrs Water Resistant 15.6 inch La...,https://www.amazon.in//Half-Moon-Waterproof-Ba...,₹649,3.9,B085MHDJ93
6,Fur Jaden 55 LTR Rucksack Travel Backpack Bag ...,https://www.amazon.in//Rucksack-Travel-Backpac...,₹849,4.3,B07YDGQQN5
7,Safari Seek 45 Ltrs Overnighter Expandable Tra...,https://www.amazon.in//Safari-45L-21-OB-BLK/dp...,"₹2,139",4.5,B07BZ5VC4H
8,FUR JADEN Anti Theft Number Lock Backpack Bag ...,https://www.amazon.in//Number-Backpack-Compart...,₹679,4.0,B09VTDMRY7
9,TRUE HUMAN EMPEROR® Anti-Theft backpack With U...,https://www.amazon.in//TRUE-HUMAN-Anti-Theft-b...,₹599,3.7,B09QJ7ZF2Y


In [8]:
# Iterate through 20 pages of product listings
product_listings = pd.DataFrame(columns=['Names', 'URLs', 'Prices', 'Ratings', 'ASIN'])

for page_num in range(1, 21):
    url = f'https://www.amazon.in/s?k=bags&page={page_num}'  # Update URL with the correct page number
    product_listings_df = scrape_product_listings(url)
    
    # Append the data to a master DataFrame
    product_listings = pd.concat([product_listings, product_listings_df], ignore_index=True)

# Export the data to CSV
product_listings.to_csv('product_listings.csv', index=False)
product_listings


,Names,URLs,Prices,Ratings,ASIN
0,"SAMRAT Laptop Backpack, Separate Compartment t...",https://www.amazon.in//sspa/click?ie=UTF8&spc=...,₹759,NaN,B0CFHCPW7V
1,"SAMRAT Laptop Backpack, Separate Compartment t...",https://www.amazon.in//sspa/click?ie=UTF8&spc=...,₹699,NaN,B0CFDSBXRY
2,Skybags Brat Black 46 Cms Casual Backpack,https://www.amazon.in//Skybags-Brat-Black-Casu...,₹669,4.1,B08Z1HHHTD
3,American Tourister Fizz Large Size 32 Ltrs Cas...,https://www.amazon.in//American-Tourister-AMT-...,"₹1,199",4.0,B07CJCGM1M
4,Half Moon Large 37L Laptop Bag Backpack for me...,https://www.amazon.in//Half-Moon-Backpack-Lugg...,₹849,3.9,B09VCLZ3K4
...,...,...,...,...,...
421,"STRABO Defender Laptop Bag, Light weight & Wat...",https://www.amazon.in//STRABO-Defender-Laptop-...,₹999,4.8,B0C9HB5SYN
422,Elegant 16 inch 28 L Casual Waterproof Laptop ...,https://www.amazon.in//sspa/click?ie=UTF8&spc=...,₹899,4.0,B0C4GWK9HR
423,Capriff Men's 35 L Casual Waterproof Laptop Ba...,https://www.amazon.in//sspa/click?ie=UTF8&spc=...,₹699,3.8,B09YRT2F7Q
424,uppercase SealPro Professional Laptop Backpack...,https://www.amazon.in//sspa/click?ie=UTF8&spc=...,"₹1,900",4.5,B0BR179L65


# Part Two

The following is the code for the part two. 
However, Amazon RESTRICTED the scraping on the site so following code could not be implemented.
'RESPONSE 503' was obtained when tried to scrape.

It should be noted that the both 2 types data tables available for product information were considered

In [423]:
# Function to scrape product details
def scrape_product_details(product_url):
    # Make a request to the product URL and parse the HTML content
    response = requests.get(product_url)
    soup  = BeautifulSoup(response, 'html.parser')
    
    # Extract description, ASIN, product description, and manufacturer
    Description, Manufacturer, Product_Description = None, None, None, 
    
    try:
        Description = soup.find('div', attrs={'id':'feature-bullets'}).text
    except:
        pass
    
    try:
        ASIN = soup.find('th',string=re.compile('ASIN')).findNext('td').text
        Manufacturer = soup.find('th',string=re.compile('Manufacturer', re.IGNORECASE)).findNext('td').text
    
    except:
        ASIN = soup.find('span', string=re.compile('ASIN')).findNext('span').text
        Manufacturer = soup.find_all('span', string=re.compile('Manufacturer', re.IGNORECASE))[-1].findNext('span').text
    
    try:
        Product_Description = soup.find(string=re.compile('Product Description', re.IGNORECASE)).parent.parent.text
    except:
        pass
    
    
    # Return the data as a dictionary
    details = {"Description": Description, 
                "ASIN": ASIN, 
                "Manufacturer": Manufacturer, 
                "Product_Description": Product_Description
                  }
    return details

In [9]:
# Iterate through product URLs and scrape details for around 200 products
product_details_df = pd.DataFrame(columns=["Description", "ASIN", "Manufacturer", "Product_Description"])

product_listings_df = pd.read_csv('product_listings.csv')
product_urls = product_listings_df['url'].values

for product_url in product_urls:
    product_data = scrape_product_details(product_url)
    
    #Append to MainDataframe
    product_data = pd.DataFrame(product_data) 
    product_details_df = product_details_df.concat([product_details_df,product_data], ignore_index=True)

In [ ]:
# Export the data to CSV
product_details_df.to_csv('product_details.csv', index=False)
